In [22]:
%pip install -q  \
               faiss-cpu tiktoken python-dotenv

In [23]:
%pip install --upgrade langchain-community langchain-openai langchain-core


In [24]:
%pip install youtube-transcript-api --upgrade


In [25]:
%pip install groq


Groq API use for LLM model


In [26]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from groq import Groq


In [ ]:
import os
os.environ["GROQ_API_KEY"] = "your_groq_api_key_here"
client = Groq(api_key=os.environ["GROQ_API_KEY"])

In [28]:
pip install sentence-transformers faiss-cpu


# Indexing (Document Ingestion)

In [29]:
from youtube_transcript_api import YouTubeTranscriptApi

video_id = "exo0OkamCnc"
try:
    transcript_list = YouTubeTranscriptApi().fetch(video_id, languages=['en'])
    texts = " ".join(chunk.text for chunk in transcript_list)
    print(texts)

except TranscriptsDisabled:
    print("No captions available for this video.")


People don't want perfect, they want authentic. People don't want flawless, they want vulnerable. People don't want polished, they want real. People don't want the highlight reel, they want the human behind it. People don't want performance. They want presence. People don't want someone who has it all together. They want someone who's willing to figure it out. Because perfection may impress, but vulnerability connects. The number one health and wellness podcast, J Shetty. J Shetty, the one, the only J Shetty. Hey everyone, it's J Shetty. Welcome back to OnPurpose. Today's episode is all about what to do when no one believes in you or supports you. The sad truth is I hear this wherever I am in the world. When I'm traveling, when I'm meeting people, when I'm connecting, one of the top things I hear is, "Jay, the people around me don't believe in me. The people around me don't support me. The people around me don't encourage me. Actually, I'm scared that if I launch my company, if I start

Step 1b - Indexing (Text Splitting)

In [30]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.create_documents([texts])
chunks_text = [chunk.page_content for chunk in chunks]


In [31]:
chunks_text = [chunk.page_content for chunk in chunks]
print(chunks_text)


['People don\'t want perfect, they want authentic. People don\'t want flawless, they want vulnerable. People don\'t want polished, they want real. People don\'t want the highlight reel, they want the human behind it. People don\'t want performance. They want presence. People don\'t want someone who has it all together. They want someone who\'s willing to figure it out. Because perfection may impress, but vulnerability connects. The number one health and wellness podcast, J Shetty. J Shetty, the one, the only J Shetty. Hey everyone, it\'s J Shetty. Welcome back to OnPurpose. Today\'s episode is all about what to do when no one believes in you or supports you. The sad truth is I hear this wherever I am in the world. When I\'m traveling, when I\'m meeting people, when I\'m connecting, one of the top things I hear is, "Jay, the people around me don\'t believe in me. The people around me don\'t support me. The people around me don\'t encourage me. Actually, I\'m scared that if I launch my c

Step - Indexing (Embedding Generation and Storing in Vector Store)

In [32]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


In [33]:
embeddings_list = model.encode(chunks_text, convert_to_numpy=True)


store in vector store

In [34]:
import faiss
import numpy as np

dimension = embeddings_list.shape[1]  # model embedding dimension
index = faiss.IndexFlatL2(dimension)  # L2 distance (cosine/Euclidean)
index.add(embeddings_list)            # add all embeddings

print(f"FAISS index ready with {index.ntotal} vectors")


FAISS index ready with 30 vectors


In [35]:
# embeddings_list already numpy array me hai
print(embeddings_list.shape)
print("First chunk embedding:")
print(embeddings_list[0])

# pehle 5 chunks
print("First 5 chunk embeddings:")
for i in range(5):
    print(embeddings_list[i])
    print("-"*50)


(30, 384)
First chunk embedding:
[ 3.99521179e-02 -4.49775569e-02  2.30948571e-02  1.27686309e-02
  3.02486140e-02  1.62519305e-03  8.36674348e-02  7.41002485e-02
  7.36312009e-03 -8.54190886e-02 -2.53022797e-02  1.83328260e-02
  5.14053889e-02 -1.10948175e-01  1.80122152e-03 -1.97965689e-02
  3.76140997e-02 -4.03971560e-02 -1.10596105e-01  3.44764441e-02
 -1.38820052e-01  8.64238571e-03 -4.39964533e-02  3.32896598e-03
 -1.05900235e-01 -4.86021891e-04 -4.85939644e-02 -4.63888943e-02
 -6.65459558e-02  1.35781961e-02  3.54299061e-02 -3.12893093e-02
  6.07994199e-02  2.02614088e-02  3.77132259e-02  3.28540020e-02
 -3.02890707e-02 -3.42160137e-03 -3.52375545e-02 -2.32198182e-02
  6.88976003e-03 -6.01421855e-02  2.46987632e-03 -1.08916890e-02
 -4.84889969e-02 -1.01887174e-01  6.75765274e-04  7.32597895e-03
  2.39090566e-02 -1.01695009e-01 -6.85466379e-02 -5.75597957e-02
  4.80503254e-02 -3.77635695e-02 -1.27755906e-02  5.71713299e-02
  4.73569613e-03  6.05274439e-02 -7.22363358e-03  2.60806

# Step 2 - Retrieval [Query + symentic search in vector ]:

In [36]:
# query = "What did the video say about AI?"
# query_vec = model.encode([query], convert_to_numpy=True)

# k = 3  # top 3 similar chunks
# distances, indices = index.search(query_vec, k)

# for idx in indices[0]:
#     print(chunks_text[idx])


Create a Context [Documents]

In [37]:
# context = "\n\n".join([chunks_text[i] for i in indices[0]])
# print(context)


# Step 3 - Augmentation
Prompting


In [38]:
# prompt = f"""
# You are a helpful assistant.
# Use the following video transcript to answer the user's question.
# if video transcript is insufficient then just say you don't know


# Context:
# {context}

# Question:
# {query}

# Answer the question in **bullet points only**, without extra explanation and Add **relevant emojis** for each point.
# """

Groq [model using
]  and Return response
😊

In [39]:
# response = client.chat.completions.create(
#     model="llama-3.3-70b-versatile",
#     messages=[
#         {"role": "user", "content": prompt}
#     ]
# )
# print("Answer:\n", response.choices[0].message.content)

# step 4. Chaining Building



In [40]:
def retriever(query, k=3):
    query_vec = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vec, k)
    return [chunks_text[i] for i in indices[0]]



def format_docs(retrieved_docs):
    # List of chunks ko ek single string me join kar diya
    return "\n\n".join(retrieved_docs)



def get_answer_from_llm(query, context):
    prompt = f"""
You are a helpful assistant.
Use the following video transcript to answer the user's question in bullet points with relevant emojis.

Context:
{context}

Question:
{query}
"""
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content



def ask_question(query, k=3):
    # Step 1: Retriever se top chunks nikalna
    retrieved_docs = retriever(query, k)

    # Step 2: Chunks ko single context me convert karna
    context = format_docs(retrieved_docs)

    # Step 3: LLM se answer lena
    answer = get_answer_from_llm(query, context)
    return answer



In [41]:
query = "What did the video say about AI?"
answer = ask_question(query)
print(answer)


* The video transcript does not mention AI at all 🤔
* It talks about public speaking, gratitude, and overcoming self-doubt 💬
* It also discusses the false consensus effect, a cognitive bias where people overestimate how much others will agree with them 📊
* The video focuses on personal growth, motivation, and the importance of perseverance 💪


In [42]:
pip freeze >requeirement.txt